In [37]:
# Matriz de custos:

cost_matrix = [[9999, 36.9, 55, 13.6], 
               [36.9, 9999, 26.5, 33.8], 
               [55, 26.5, 9999, 45.8], 
               [13.6, 33.8, 45.8, 9999]]

In [38]:
# Número de pontos:

n = len(cost_matrix)
n

4

In [39]:
# Formulação do modelo:

import pyomo.environ as pyo

# Objeto modelo
model = pyo.ConcreteModel()

# Índices dos pontos (1,...,n)
model.M = pyo.RangeSet(n)                
model.N = pyo.RangeSet(n)

# Índice da variável U (2,...,n)
model.U = pyo.RangeSet(2,n)

# Variáveis de decisão xij
model.x = pyo.Var(model.N,model.M, within=pyo.Binary)

# Variávei ui
model.u = pyo.Var(model.N, within=pyo.NonNegativeIntegers,bounds=(0,n-1))

# Matriz de custos
model.c = pyo.Param(model.N, model.M,initialize=lambda model, i, j: cost_matrix[i-1][j-1])

# Função objetivo
def obj_func(model):
    return sum(model.x[i,j] * model.c[i,j] for i in model.N for j in model.M)

model.objective = pyo.Objective(rule=obj_func,sense=pyo.minimize)

# Restrições 1
def rule_const1(model,M):
    return sum(model.x[i,M] for i in model.N if i!=M ) == 1

model.const1 = pyo.Constraint(model.M,rule=rule_const1)


# Restrições 2
def rule_const2(model,N):
    return sum(model.x[N,j] for j in model.M if j!=N) == 1

model.rest2 = pyo.Constraint(model.N,rule=rule_const2)


# Restrições MTZ
def rule_const3(model,i,j):
    if i!=j: 
        return model.u[i] - model.u[j] + model.x[i,j] * n <= n-1
    else:
        return model.u[i] - model.u[i] == 0 
    
model.rest3 = pyo.Constraint(model.U,model.N,rule=rule_const3)

# model.pprint()
# pprint(filename=’file.txt’)

In [45]:
# 100 execuções para obter o tempo médio:

t = []
for i in range(100):
    solver = pyo.SolverFactory('glpk')
    result = solver.solve(model,tee = False)
    t.append(result.Solver.Time)

print(result)


Problem: 
- Name: unknown
  Lower bound: 122.8
  Upper bound: 122.8
  Number of objectives: 1
  Number of constraints: 21
  Number of variables: 21
  Number of nonzeros: 52
  Sense: minimize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 5
      Number of created subproblems: 5
  Error rc: 0
  Time: 0.06083846092224121
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



In [47]:
from statistics import *

# Tempo médio de execução
print(mean(t), stdev(t))

0.05828287363052368 0.005580330615349053


In [41]:
l = list(model.x.keys())
for i in l:
    if model.x[i]() != 0:
        print(i,'--', model.x[i]())

(1, 2) -- 1.0
(2, 3) -- 1.0
(3, 4) -- 1.0
(4, 1) -- 1.0
